In [1]:
pip install tweepy

In [2]:
import tweepy
import pandas as pd

In [7]:
import time
import tweepy
import pandas as pd

# Provide your Bearer Token
bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKwyAEAAAAAHWRWJIG0fcNZlFYbBSIrEzYclm0%3DQdb24tcUzU8Ys502eSnymYi9jg8EHFwmZfrPiVdfjIiwv1dTkk"

# Initialize Tweepy Client
client = tweepy.Client(bearer_token=bearer_token)

# Define query for #Costco
query = "#Costco -is:retweet"

# Create a list to store the data
tweets_data = []

# Track the total number of requests made
total_requests = 0
max_requests = 100  # Set your monthly request limit

try:
    # Fetch 5 tweets
    if total_requests < max_requests:
        response = client.search_recent_tweets(
            query=query,
            max_results=5,  # Fetch only 5 tweets
            tweet_fields=["created_at", "author_id", "conversation_id", "public_metrics", "lang"]
        )
        total_requests += 1  # Increment requests for the tweet search

        if response.data:
            for tweet in response.data:
                tweet_id = tweet.id
                conversation_id = tweet.conversation_id  # Replies are in the same conversation_id
                author_id = tweet.author_id
                text = tweet.text
                created_at = tweet.created_at

                # Add the original tweet to the dataset
                tweets_data.append({
                    "tweet_id": tweet_id,
                    "conversation_id": conversation_id,
                    "author_id": author_id,
                    "created_at": created_at,
                    "text": text,
                    "is_reply": False,  # Mark it as the original tweet
                })

                # Fetch up to 5 replies for this tweet if under request limit
                if total_requests < max_requests:
                    time.sleep(1)  # Add a delay to avoid rapid-fire requests
                    reply_query = f"conversation_id:{conversation_id} -is:retweet"
                    reply_response = client.search_recent_tweets(
                        query=reply_query,
                        max_results=5,  # Fetch up to 5 replies per conversation
                        tweet_fields=["created_at", "author_id", "conversation_id"]
                    )
                    total_requests += 1  # Increment requests for each reply search

                    if reply_response.data:
                        for reply in reply_response.data:
                            # Avoid re-adding the original tweet
                            if reply.id == tweet_id:
                                continue

                            tweets_data.append({
                                "tweet_id": reply.id,
                                "conversation_id": reply.conversation_id,
                                "author_id": reply.author_id,
                                "created_at": reply.created_at,
                                "text": reply.text,
                                "is_reply": True,  # Mark it as a reply
                            })

except tweepy.TooManyRequests as e:
    reset_time = int(e.response.headers.get("x-rate-limit-reset", 0))
    wait_time = max(0, reset_time - int(time.time()))  # Calculate wait time
    print(f"Rate limit hit. Sleeping for {wait_time} seconds...")
    time.sleep(wait_time + 1)  # Sleep until the rate limit resets
    print("Resuming...")

# Save the data to a CSV file
df = pd.DataFrame(tweets_data)
df.to_csv("tweets_costco_limited.csv", index=False)
print(f"Saved data to tweets_costco_limited.csv. Total requests used: {total_requests}")


Rate limit hit. Sleeping for 569 seconds...
Resuming...
Saved data to tweets_costco_limited.csv. Total requests used: 0
